# Fornite Ranking 
Dataset created with API and WebScrapping

In [1]:
import os
import requests
import pandas as pd
import numpy as np
from dotenv import load_dotenv, find_dotenv
from bs4 import BeautifulSoup

## Web Scraping to get the top 100 players - Fortnite

In [2]:
#Web Scraping to get the top 100 Fortnite players (with most money)
response= requests.get('https://www.esportsearnings.com/games/534-fortnite/top-players')
html = response.content
soup = BeautifulSoup(html, "lxml")
table_100 = pd.read_html(html, attrs={'class' : 'detail_list_table'})[0]

## Web Scraping to get the top 101 to 200 players - Fortnite

In [3]:
#Web Scraping to get the top 101-200 Fortnite players (with most money)
response_200= requests.get('https://www.esportsearnings.com/games/534-fortnite/top-players-x100')
html_200 = response_200.content
soup_200 = BeautifulSoup(html, "lxml")
table_200 = pd.read_html(html_200, attrs={'class' : 'detail_list_table'})[0]

## Concatenate to get TOP 200

In [4]:
top200 = pd.concat([table_100, table_200]).reset_index(drop=True )
pd.set_option('display.max_rows', None)

In [5]:
#Create another column called "Name Correct" to enable the correct merge
top200['Name Correct'] =[str.capitalize(i) for i in top200['Player ID']]

In [6]:
top200

,Unnamed: 0,Player ID,Player Name,Total (Game),Total (Overall),% of Total,Name Correct
0,1.0,Bugha,Kyle Giersdorf,"$3,072,333.34","$3,072,333.34",100.00%,Bugha
1,2.0,Aqua,David Wang,"$1,900,941.23","$1,900,941.23",100.00%,Aqua
2,3.0,psalm,Harrison Chang,"$1,870,738.80","$1,954,950.00",95.69%,Psalm
3,4.0,Nyhrox,Emil Bergquist Pedersen,"$1,533,345.69","$1,533,345.69",100.00%,Nyhrox
4,5.0,EpikWhale,Shane Cotton,"$1,330,841.54","$1,330,841.54",100.00%,Epikwhale
5,6.0,Wolfiez,Jaden Ashman,"$1,315,726.67","$1,315,726.67",100.00%,Wolfiez
6,7.0,Rojo,Dave Jong,"$1,208,143.33","$1,208,143.33",100.00%,Rojo
7,8.0,Kreo,Nate Kou,"$1,180,150.00","$1,180,150.00",100.00%,Kreo
8,9.0,Zayt,Williams Aubin,"$1,122,941.53","$1,123,666.53",99.94%,Zayt
9,10.0,Ceice,Davis McClellan,"$1,102,213.80","$1,102,213.80",100.00%,Ceice


## API to get the information about top players
Source: https://dash.fortnite-api.com/

In [7]:
#Applying the API for each Player ID
url = [f'https://fortnite-api.com/v1/stats/br/v2?name={name}' for name in top200['Player ID']]
response = [requests.get(i).json() for i in url]

In [8]:
#Turn into a dataframe
x = pd.json_normalize(response)
pd.set_option('display.max_columns', None)

# Merge informations

In [9]:
#Merge the scrap with API content
all_info = pd.merge(left=top200, right=x, left_on='Name Correct', right_on='data.account.name')

In [10]:
#Data Cleaning
all_info.drop(['status','error', 'Name Correct','data.image'], inplace = True, axis = 1)
all_info.rename(columns = {'Unnamed: 0':'Rank Position'}, inplace = True)
columns_to_drop = list(all_info.loc[: ,all_info.isnull().mean()>0.8].columns)
all_info.drop(columns = columns_to_drop, inplace = True)

In [11]:
#Data Cleaning - Part 02
columns_to_remove = list(all_info.loc[:,[i.startswith(' stats') for i in list(all_info.columns)]].columns)
all_info.drop(columns = columns_to_remove, inplace = True)

In [12]:
#Renaming Columns
rename_columns = [(' '.join(i.split(sep='.'))).replace('data stats all','') for i in all_info.columns]
rename_columns2 = [i.replace('data','') for i in rename_columns]
all_info.columns= rename_columns2

In [13]:
all_info

,Rank Position,Player ID,Player Name,Total (Game),Total (Overall),% of Total,account id,account name,battlePass level,battlePass progress,overall score,overall scorePerMin,overall scorePerMatch,overall wins,overall top3,overall top5,overall top6,overall top10,overall top12,overall top25,overall kills,overall killsPerMin,overall killsPerMatch,overall deaths,overall kd,overall matches,overall winRate,overall minutesPlayed,overall playersOutlived,overall lastModified,solo score,solo scorePerMin,solo scorePerMatch,solo wins,solo top10,solo top25,solo kills,solo killsPerMin,solo killsPerMatch,solo deaths,solo kd,solo matches,solo winRate,solo minutesPlayed,solo playersOutlived,solo lastModified,duo score,duo scorePerMin,duo scorePerMatch,duo wins,duo top5,duo top12,duo kills,duo killsPerMin,duo killsPerMatch,duo deaths,duo kd,duo matches,duo winRate,duo minutesPlayed,duo playersOutlived,duo lastModified,trio score,trio scorePerMin,trio scorePerMatch,trio wins,trio top3,trio top6,trio kills,trio killsPerMin,trio killsPerMatch,trio deaths,trio kd,trio matches,trio winRate,trio minutesPlayed,trio playersOutlived,trio lastModified,squad score,squad scorePerMin,squad scorePerMatch,squad wins,squad top3,squad top6,squad kills,squad killsPerMin,squad killsPerMatch,squad deaths,squad kd,squad matches,squad winRate,squad minutesPlayed,squad playersOutlived,squad lastModified,ltm score,ltm scorePerMin,ltm scorePerMatch,ltm wins,ltm kills,ltm killsPerMin,ltm killsPerMatch,ltm deaths,ltm kd,ltm matches,ltm winRate,ltm minutesPlayed,ltm playersOutlived,ltm lastModified,stats keyboardMouse overall score,stats keyboardMouse overall scorePerMin,stats keyboardMouse overall scorePerMatch,stats keyboardMouse overall wins,stats keyboardMouse overall top3,stats keyboardMouse overall top5,stats keyboardMouse overall top6,stats keyboardMouse overall top10,stats keyboardMouse overall top12,stats keyboardMouse overall top25,stats keyboardMouse overall kills,stats keyboardMouse overall killsPerMin,stats keyboardMouse overall killsPerMatch,stats keyboardMouse overall deaths,stats keyboardMouse overall kd,stats keyboardMouse overall matches,stats keyboardMouse overall winRate,stats keyboardMouse overall minutesPlayed,stats keyboardMouse overall playersOutlived,stats keyboardMouse overall lastModified,stats keyboardMouse solo score,stats keyboardMouse solo scorePerMin,stats keyboardMouse solo scorePerMatch,stats keyboardMouse solo wins,stats keyboardMouse solo top10,stats keyboardMouse solo top25,stats keyboardMouse solo kills,stats keyboardMouse solo killsPerMin,stats keyboardMouse solo killsPerMatch,stats keyboardMouse solo deaths,stats keyboardMouse solo kd,stats keyboardMouse solo matches,stats keyboardMouse solo winRate,stats keyboardMouse solo minutesPlayed,stats keyboardMouse solo playersOutlived,stats keyboardMouse solo lastModified,stats keyboardMouse duo score,stats keyboardMouse duo scorePerMin,stats keyboardMouse duo scorePerMatch,stats keyboardMouse duo wins,stats keyboardMouse duo top5,stats keyboardMouse duo top12,stats keyboardMouse duo kills,stats keyboardMouse duo killsPerMin,stats keyboardMouse duo killsPerMatch,stats keyboardMouse duo deaths,stats keyboardMouse duo kd,stats keyboardMouse duo matches,stats keyboardMouse duo winRate,stats keyboardMouse duo minutesPlayed,stats keyboardMouse duo playersOutlived,stats keyboardMouse duo lastModified,stats keyboardMouse trio score,stats keyboardMouse trio scorePerMin,stats keyboardMouse trio scorePerMatch,stats keyboardMouse trio wins,stats keyboardMouse trio top3,stats keyboardMouse trio top6,stats keyboardMouse trio kills,stats keyboardMouse trio killsPerMin,stats keyboardMouse trio killsPerMatch,stats keyboardMouse trio deaths,stats keyboardMouse trio kd,stats keyboardMouse trio matches,stats keyboardMouse trio winRate,stats keyboardMouse trio minutesPlayed,stats keyboardMouse trio playersOutlived,stats keyboardMouse trio lastModified,stats keyboardMouse squad score,stats keyboardMouse squad sc

In [14]:
#Top players - Export to CSV
all_info.to_csv('./Datasets/top_players.csv', index=False)

# Using the API to get friends information

In [15]:
friends=['Floelr', 'Monsterbale', 'Sharshock','BEP Rodolfo477','williamtotake','BzAlva','Cauanyca','Guibru123']
results = [requests.get(f'https://fortnite-api.com/v1/stats/br/v2?name={i}').json() for i in friends]

In [16]:
friends_compare= pd.json_normalize(results)

In [17]:
#Data Cleaning
columns_to_drop = list(friends_compare.loc[: ,friends_compare.isnull().mean()>0.8].columns)
friends_compare.drop(columns=columns_to_drop, inplace=True)

In [18]:
#Data Cleaning - Part2
columns_to_remove = list(friends_compare.loc[:,[i.startswith(' stats') for i in list(friends_compare.columns)]].columns)
friends_compare.drop(columns = columns_to_remove, inplace = True)
friends_compare.drop(columns=['status'], inplace=True)

In [19]:
#Renaming Columns
rename_columns = [(' '.join(i.split(sep='.'))).replace('data stats all','') for i in friends_compare.columns]
rename_columns2 = [i.replace('data','') for i in rename_columns]
friends_compare.columns= rename_columns2

In [20]:
friends_compare

,account id,account name,battlePass level,battlePass progress,overall score,overall scorePerMin,overall scorePerMatch,overall wins,overall top3,overall top5,overall top6,overall top10,overall top12,overall top25,overall kills,overall killsPerMin,overall killsPerMatch,overall deaths,overall kd,overall matches,overall winRate,overall minutesPlayed,overall playersOutlived,overall lastModified,solo score,solo scorePerMin,solo scorePerMatch,solo wins,solo top10,solo top25,solo kills,solo killsPerMin,solo killsPerMatch,solo deaths,solo kd,solo matches,solo winRate,solo minutesPlayed,solo playersOutlived,solo lastModified,duo score,duo scorePerMin,duo scorePerMatch,duo wins,duo top5,duo top12,duo kills,duo killsPerMin,duo killsPerMatch,duo deaths,duo kd,duo matches,duo winRate,duo minutesPlayed,duo playersOutlived,duo lastModified,trio score,trio scorePerMin,trio scorePerMatch,trio wins,trio top3,trio top6,trio kills,trio killsPerMin,trio killsPerMatch,trio deaths,trio kd,trio matches,trio winRate,trio minutesPlayed,trio playersOutlived,trio lastModified,squad score,squad scorePerMin,squad scorePerMatch,squad wins,squad top3,squad top6,squad kills,squad killsPerMin,squad killsPerMatch,squad deaths,squad kd,squad matches,squad winRate,squad minutesPlayed,squad playersOutlived,squad lastModified,ltm score,ltm scorePerMin,ltm scorePerMatch,ltm wins,ltm kills,ltm killsPerMin,ltm killsPerMatch,ltm deaths,ltm kd,ltm matches,ltm winRate,ltm minutesPlayed,ltm playersOutlived,ltm lastModified,stats gamepad overall score,stats gamepad overall scorePerMin,stats gamepad overall scorePerMatch,stats gamepad overall wins,stats gamepad overall top3,stats gamepad overall top5,stats gamepad overall top6,stats gamepad overall top10,stats gamepad overall top12,stats gamepad overall top25,stats gamepad overall kills,stats gamepad overall killsPerMin,stats gamepad overall killsPerMatch,stats gamepad overall deaths,stats gamepad overall kd,stats gamepad overall matches,stats gamepad overall winRate,stats gamepad overall minutesPlayed,stats gamepad overall playersOutlived,stats gamepad overall lastModified,stats gamepad solo score,stats gamepad solo scorePerMin,stats gamepad solo scorePerMatch,stats gamepad solo wins,stats gamepad solo top10,stats gamepad solo top25,stats gamepad solo kills,stats gamepad solo killsPerMin,stats gamepad solo killsPerMatch,stats gamepad solo deaths,stats gamepad solo kd,stats gamepad solo matches,stats gamepad solo winRate,stats gamepad solo minutesPlayed,stats gamepad solo playersOutlived,stats gamepad solo lastModified,stats gamepad duo score,stats gamepad duo scorePerMin,stats gamepad duo scorePerMatch,stats gamepad duo wins,stats gamepad duo top5,stats gamepad duo top12,stats gamepad duo kills,stats gamepad duo killsPerMin,stats gamepad duo killsPerMatch,stats gamepad duo deaths,stats gamepad duo kd,stats gamepad duo matches,stats gamepad duo winRate,stats gamepad duo minutesPlayed,stats gamepad duo playersOutlived,stats gamepad duo lastModified,stats gamepad trio score,stats gamepad trio scorePerMin,stats gamepad trio scorePerMatch,stats gamepad trio wins,stats gamepad trio top3,stats gamepad trio top6,stats gamepad trio kills,stats gamepad trio killsPerMin,stats gamepad trio killsPerMatch,stats gamepad trio deaths,stats gamepad trio kd,stats gamepad trio matches,stats gamepad trio winRate,stats gamepad trio minutesPlayed,stats gamepad trio playersOutlived,stats gamepad trio lastModified,stats gamepad squad score,stats gamepad squad scorePerMin,stats gamepad squad scorePerMatch,stats gamepad squad wins,stats gamepad squad top3,stats gamepad squad top6,stats gamepad squad kills,stats gamepad squad killsPerMin,stats gamepad squad killsPerMatch,stats gamepad squad deaths,stats gamepad squad kd,stats gamepad squad matches,stats gamepad squad winRate,stats gamepad squad minutesPlayed,stats gamepad squad playersOutlived,stats gamepad squad lastModified,stats gamepad ltm score,stats gamepad ltm scorePerMin,stats gamepad ltm scor

In [22]:
#Top friends - Export to CSV
friends_compare.to_csv('./Datasets/top_friends.csv', index=False)

## Concat friends & pro-player information

In [23]:
gather = pd.concat([all_info, friends_compare])
gather.reset_index(drop=True,inplace=True)
gather.drop(columns=['Rank Position','Player ID','Player Name', 'Total (Game)','Total (Overall)','% of Total'], inplace=True)

In [24]:
columns_to_drop = list(gather.loc[: ,gather.isnull().mean()>0.8].columns)
gather.drop(columns=columns_to_drop, inplace=True)
columns_to_remove = list(gather.loc[:,[i.startswith(' stats') for i in list(gather.columns)]].columns)
gather.drop(columns = columns_to_remove, inplace = True)

In [25]:
gather

,account id,account name,battlePass level,battlePass progress,overall score,overall scorePerMin,overall scorePerMatch,overall wins,overall top3,overall top5,overall top6,overall top10,overall top12,overall top25,overall kills,overall killsPerMin,overall killsPerMatch,overall deaths,overall kd,overall matches,overall winRate,overall minutesPlayed,overall playersOutlived,overall lastModified,solo score,solo scorePerMin,solo scorePerMatch,solo wins,solo top10,solo top25,solo kills,solo killsPerMin,solo killsPerMatch,solo deaths,solo kd,solo matches,solo winRate,solo minutesPlayed,solo playersOutlived,solo lastModified,duo score,duo scorePerMin,duo scorePerMatch,duo wins,duo top5,duo top12,duo kills,duo killsPerMin,duo killsPerMatch,duo deaths,duo kd,duo matches,duo winRate,duo minutesPlayed,duo playersOutlived,duo lastModified,trio score,trio scorePerMin,trio scorePerMatch,trio wins,trio top3,trio top6,trio kills,trio killsPerMin,trio killsPerMatch,trio deaths,trio kd,trio matches,trio winRate,trio minutesPlayed,trio playersOutlived,trio lastModified,squad score,squad scorePerMin,squad scorePerMatch,squad wins,squad top3,squad top6,squad kills,squad killsPerMin,squad killsPerMatch,squad deaths,squad kd,squad matches,squad winRate,squad minutesPlayed,squad playersOutlived,squad lastModified,ltm score,ltm scorePerMin,ltm scorePerMatch,ltm wins,ltm kills,ltm killsPerMin,ltm killsPerMatch,ltm deaths,ltm kd,ltm matches,ltm winRate,ltm minutesPlayed,ltm playersOutlived,ltm lastModified
0,121afd34c94d4a6eae6d2aa305452a1a,Aqua,1.0,0.0,203707.0,23.067,60.251,52.0,78.0,77.0,172.0,56.0,234.0,139.0,2918.0,0.330,0.863,3329.0,0.877,3381.0,1.538,8831.0,67849.0,2020-09-09T21:58:53Z,56772.0,23.373,65.330,8.0,52.0,132.0,740.0,0.305,0.852,861.0,0.859,869.0,0.921,2429.0,21476.0,2020-08-18T05:47:23Z,92314.0,23.177,76.546,17.0,73.0,217.0,1006.0,0.253,0.834,1189.0,0.846,1206.0,1.410,3983.0,29673.0,2020-09-09T21:58:53Z,2759.0,23.784,153.278,1.0,1.0,3.0,21.0,0.181,1.167,17.0,1.235,18.0,5.556,116.0,773.0,2019-09-19T23:58:54Z,51862.0,22.519,40.266,26.0,60.0,140.0,1151.0,0.500,0.894,1262.0,0.912,1288.0,2.019,2303.0,15927.0,2020-08-12T01:20:57Z,156911.0,38.984,218.235,45.0,1263.0,0.314,1.757,674.0,1.874,719.0,6.259,4025.0,12108.0,2020-09-09T22:14:51Z
1,5ca459a055b74b68bd6cd661fcfe0021,Psalm,1.0,0.0,869135.0,22.708,104.225,239.0,266.0,245.0,465.0,602.0,550.0,1285.0,10111.0,0.264,1.212,8100.0,1.248,8339.0,2.866,38275.0,250351.0,2020-09-13T01:22:59Z,448864.0,23.027,108.921,85.0,587.0,1262.0,4774.0,0.245,1.158,4036.0,1.183,4121.0,2.063,19493.0,131149.0,2020-09-13T00:07:56Z,193646.0,22.622,91.558,67.0,218.0,507.0,2281.0,0.266,1.078,2048.0,1.114,2115.0,3.168,8560.0,58685.0,2020-09-12T23:33:40Z,91710.0,19.774,195.544,15.0,61.0,117.0,659.0,0.142,1.405,454.0,1.452,469.0,3.198,4638.0,23243.0,2020-08-26T03:42:29Z,134915.0,24.161,82.567,72.0,159.0,281.0,2397.0,0.429,1.467,1562.0,1.535,1634.0,4.406,5584.0,37274.0,2020-09-13T01:22:59Z,100855.0,28.434,131.837,49.0,1532.0,0.432,2.003,716.0,2.140,765.0,6.405,3547.0,19375.0,2020-08-10T01:00:49Z
2,37ed3f19bcc84880a75bbcb98a775a46,Wolfiez,1.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,1.0,0.000,1.0,0.000,0.0,0.0,1970-01-01T00:00:00Z,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.000,0.000,1.0,0.000,1.0,0.000,0.0,0.0,1970-01-01T00:00:00Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2aed14dea20c4d51aff3f2d6087f8e22,Rojo,1.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.000,0.111,9.0,0.111,9.0,0.000,0.0,0.0,1970-01-01T00:00:00Z,0.0,0.000,0.000,0.0,0.0,1.0,1.0,0.000,0.167,6.0,0.167,6.0,0.000,0.0,0.0,1970-01-01T00:00:00Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.000,0.000,3.0,0.000,3.0,0.000,0.0,0.

In [26]:
#Top Players and Friends
gather.to_csv('./Datasets/top_and_friends.csv', index=False)

## Top 100 Highest Overall Earnings

In [27]:
#Web-Scrap: Top 100 Highest Overall Earnings
response= requests.get('https://www.esportsearnings.com/players/highest-overall')
html = response.content
soup = BeautifulSoup(html, "lxml")
table = pd.read_html(html, attrs={'class' : 'detail_list_table'})[0]

In [28]:
# Replacing Header with Top Row - Just run once
new_header = table.iloc[0] 
table = table[1:] 
table.columns = new_header

In [29]:
table.rename(columns={'NaN':'Rank Position'}, inplace=True)

In [30]:
#Top 100 Highest Overall Earnings - Export csv
table.to_csv('./Datasets/highest_earnings_all_games.csv', index=False)

In [31]:
table #Top 100 Highest Overall Earnings

,NaN,Player ID,Player Name,Total (Overall),Highest Paying Game,Total (Game),% of Total
1,1.0,N0tail,Johan Sundstein,"$6,940,222.80",Dota 2,"$6,928,496.58",99.83%
2,2.0,JerAx,Jesse Vainikka,"$6,470,548.78",Dota 2,"$6,470,000.02",99.99%
3,3.0,ana,Anathan Pham,"$6,000,411.96",Dota 2,"$6,000,411.96",100.00%
4,4.0,Ceb,SÃ©bastien Debs,"$5,530,197.41",Dota 2,"$5,530,197.41",100.00%
5,5.0,Topson,Topias Taavitsainen,"$5,448,502.57",Dota 2,"$5,448,502.57",100.00%
6,6.0,KuroKy,Kuro Takhasomi,"$5,190,106.15",Dota 2,"$5,188,382.81",99.97%
7,7.0,Miracle-,Amer Al-Barkawi,"$4,789,911.48",Dota 2,"$4,789,911.48",100.00%
8,8.0,MinD_ContRoL,Ivan Ivanov,"$4,570,985.96",Dota 2,"$4,570,985.96",100.00%
9,9.0,GH,Maroun Merhej,"$4,183,919.04",Dota 2,"$4,183,919.04",100.00%
10,10.0,Matumbaman,Lasse Urpalainen,"$3,740,194.44",Dota 2,"$3,740,194.44",100.00%


## Top 100 Highest Overall Earnings - Brazil

In [32]:
#Top 100 Highest Earnings for Brazil
response= requests.get('https://www.esportsearnings.com/countries/br')
html = response.content
soup = BeautifulSoup(html, "lxml")
table2 = pd.read_html(html, attrs={'class' : 'detail_list_table'})[0]
#Turning the first row into header
new_header = table2.iloc[0] 
table2 = table2[1:] 
table2.columns = new_header

In [33]:
table2

,NaN,Player ID,Player Name,Total (Overall),Highest Paying Game,Total (Game),% of Total
1,1.0,FalleN,Gabriel Toledo,"$1,067,842.41",Counter-Strike: Global Offensive,"$1,059,938.92",99.26%
2,2.0,TACO,EpitÃ¡cio de Melo,"$1,063,858.27",Counter-Strike: Global Offensive,"$1,063,858.27",100.00%
3,3.0,fer,Fernando Alvarenga,"$1,054,943.90",Counter-Strike: Global Offensive,"$1,054,038.92",99.91%
4,4.0,coldzera,Marcelo David,"$1,002,401.46",Counter-Strike: Global Offensive,"$1,002,401.46",100.00%
5,5.0,fnx,Lincoln Lau,"$424,984.20",Counter-Strike: Global Offensive,"$386,448.82",90.93%
6,6.0,boltz,Ricardo Prass,"$368,373.62",Counter-Strike: Global Offensive,"$368,373.62",100.00%
7,7.0,PVDDR,Paulo Damo da Rosa,"$337,800.00",Magic: The Gathering Arena,"$337,800.00",100.00%
8,8.0,felps,JoÃ£o Vasconcellos,"$318,291.87",Counter-Strike: Global Offensive,"$318,291.87",100.00%
9,9.0,GuiFera,Guilherme Fonseca,"$237,251.21",Pro Evolution Soccer 2017,"$206,000.00",86.83%
10,10.0,hen1,Henrique Teles,"$225,379.43",Counter-Strike: Global Offensive,"$224,474.45",99.60%


In [34]:
#Top 100 Highest Earnings for Brazil - Export csv
table2.to_csv('./Datasets/highest_earnings_all_games_brazil.csv', index=False)

## Insights:
    - Compare if the player which wins more money is the player with more kills per minute; 
    - Compare if the player which wins more money is the player with more kills per match;
    - Analysing how many players with earns more money, earn it playing fortnite;
    - Compare wich of my friends kills more, and compare us with pro players;
    - Analyse how many brazilians it's on the top 100 of the world;